In [ ]:
import pupil_classv2 as pc
import math
from statistics import mean
import csv
from timestamps import new_timestamps
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from helper import *
import tdt
from scipy.signal import find_peaks, peak_widths,resample,correlate, correlation_lags,peak_prominences,resample,butter, lfilter, freqz,medfilt
from scipy.stats import pearsonr,wilcoxon,mannwhitneyu,kruskal
import os
from fp_helper import *
from matplotlib import rcParams
import cv2
from scipy.stats import ttest_ind,ttest_rel,zscore
from sklearn.linear_model import Ridge,RidgeCV
from sklearn.model_selection import KFold,cross_val_score
from random import randint
from scipy.ndimage import median_filter
import scikit_posthocs as sp

params = {
        "font.family" : "Arial",
        'pdf.fonttype' : 42,
        'axes.labelsize': 10,
        'axes.titlesize': 11,
        'axes.linewidth': 0.5,
        'xtick.labelsize':9,
        'xtick.major.width':0.5,
        'ytick.major.width':0.5,
        'ytick.labelsize':9,
        'axes.spines.top':False,
        'axes.spines.right':False
         }
rcParams['figure.figsize'] = 21.7,8.27
rcParams.update(params)
def format_ax(ax,xlim,ylim,xspace,yspace):
   

    ax.set_xticks(np.arange(xlim[0],xlim[1]+ xspace,xspace))
    ax.set_yticks(np.arange(ylim[0],ylim[1]+ yspace,yspace))

    ax.set(ylim=(ylim[0], ylim[1]))
    ax.set(xlim=(xlim[0], xlim[1]))


In [ ]:
###low pass filter pupil
def filter_pupil(pupil,order = 1,cutoff = 1,fs = 20):
    def butter_lowpass(cutoff, fs, order=5):
        return butter(order, cutoff, fs=fs, btype='low', analog=False)

    def butter_lowpass_filter(data, cutoff, fs, order=5):
        b, a = butter_lowpass(cutoff, fs, order=order)
        y = lfilter(b, a, data)
        return y
    return butter_lowpass_filter(pupil, cutoff, fs, order)

In [ ]:
### get pupil events
###get events from all sessions

def get_pupil_events(id,date):
    id = id
    date = date
    face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
    path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
    df = pd.read_csv(path)
    pupil = np.array(df["Pupil Size"])
    time = np.array(df["Time"])
    speed = np.array(df["Running Speed"])
    speed[0] = 0
    dff = np.array(df["dFF"])
    face = np.array(pd.read_csv(face_path)["Facial Movement"])

    ###Onset detection alogrithm

    
    filter = filter_pupil(pupil) ###first apply low pass filter to pupil trace

    def groupSequence(x): 
        it = iter(x) 
        prev, res = next(it), [] 
    
        while prev is not None: 
            start = next(it, None) 
    
            if start and start > prev:

                res.append(prev) 
            elif res: 
                yield list(res + [prev]) 
                res = [] 
            prev = start 


    regions = list(groupSequence(filter))
    corrected_regions = [region for region in regions if len(region) >=15 ] ###only include regions that are >750ms
    corrected_regions = [region for region in corrected_regions if region[0]<np.median(filter) ] ###only include onsets that occur below z = median

    region_ix = [[np.where(filter == val)[0][0] for val in region] for region in corrected_regions] ### get indices for each region


    ##join regions that are near eachother
    new = []
    for i,first in enumerate(region_ix):
        if i < len(region_ix)-1:
            second = region_ix[i + 1]
            end = first[-1]
            start = second[0]
            if first[0] - region_ix[i - 1][-1]>20:
                if start-end<= 20:
                    #print(first[0]/20,second[0]/20)
                    new.append(first + second)
                else:
                    new.append(first)
    region_ix = new
    corrected_regions = [filter[region[0]:region[-1]] for region in region_ix]


    new = []
    for i,first in enumerate(region_ix):
        if i < len(region_ix)-1:
            second = region_ix[i + 1]
            end = first[-1]
            start = second[0]
            if first[0] - region_ix[i - 1][-1]>10:
                if start-end<= 10:
                    #print(first[0]/20,second[0]/20)
                    new.append(first + second)
                else:
                    new.append(first)
    region_ix = new
    corrected_regions = [filter[region[0]:region[-1]] for region in region_ix]
    onsets_ix = [region[0] for region in region_ix]


    ###get offsets and durations for pupil event
    offset_ix = []
    durations = []
    for i,(region,index) in enumerate(zip(corrected_regions,region_ix)):
        if i < len(region_ix)-1:
            start = index[-1] ###end of region
            end = region_ix[i+1][0] ### start of next region
            onset_y = region[0]
            offsets = np.where(filter[start:end] <= onset_y) ###find ix of first point that goes below onset 
        if len(offsets[0]) == 0: ###if it never goes below onset then use lowest value
            offset = start + np.argmin(filter[start:end])
        else:
            offset = start + offsets[0][0]


        offset_ix.append(offset)

        durations.append((offset-index[0])/20)

    ###handle last pupil event
    start = region_ix[-1][-1]
    end = 36000
    onset_y = corrected_regions[-1][0]
    offsets = np.where(filter[start:end] <= onset_y)
    if len(offsets[0]) == 0:
        offset = end-1
    else:
        offset = start + np.argmin(filter[start:end])

    offset_ix[-1] = offset

    durations[-1] = (offset-region_ix[-1][0])/20

    ###get pupil event amplitudes
    amplitudes = []
    amplitudes_ix = []
    for on,off in zip(onsets_ix,offset_ix):
        #print(on,off)
        amplitudes.append(max(filter[on:off])-filter[on])
        amplitudes_ix.append(on + np.argmax(filter[on:off]))

    ###get pupil event slopes
    slopes = []
    for region in corrected_regions:
        run = len(region)/20
        rise = region[-1]-region[0]
        slopes.append(rise/run)

    ###get dff data
    aucs = []
    peak_dffs = []
    mean_dffs = []
    for onset,offset in zip(onsets_ix,offset_ix):
        baseline = np.mean(dff[onset-40:onset-10])
        peak_dffs.append(max(dff[onset:offset]) - np.mean(dff[onset-30:onset]))
        mean_dffs.append(np.mean(dff[onset:offset])- np.mean(dff[onset-20:onset]))

    ###create dictionary

    events_dict = {"durations" :durations,
    "amplitudes": amplitudes,
    "slopes" : slopes,
    "onsets" : onsets_ix,
    "offsets" : offset_ix,
    "peak_ix" : amplitudes_ix,
    "ID" : [id] * len(durations),
    "Date" : [date] * len(durations),
    "peak_dff" : peak_dffs,
    "mean_dff" : mean_dffs,


    }

    return pd.DataFrame.from_dict(events_dict)


In [ ]:
###get quiet regions
def contiguous_regions(condition):
    """Finds contiguous True regions of the boolean array "condition". Returns
    a 2D array where the first column is the start index of the region and the
    second column is the end index."""

    # Find the indicies of changes in "condition"
    d = np.diff(condition)
    idx, = d.nonzero() 

    # We need to start things after the change in "condition". Therefore, 
    # we'll shift the index by 1 to the right.
    idx += 1

    if condition[0]:
        # If the start of condition is True prepend a 0
        idx = np.r_[0, idx]

    if condition[-1]:
        # If the end of condition is True, append the length of the array
        idx = np.r_[idx, condition.size] # Edit

    # Reshape the result into two columns
    idx.shape = (-1,2)
    return idx




def get_movement_events(events,speed):
    onsets = [on for on in events["onsets"]] ###get pupil onsets ix
    offsets = [off for off in events["offsets"]] ###get pupil offsets ix
    events_ix = []
    for i,(on,off) in enumerate(zip(onsets,offsets)):
        if np.mean(speed[on:off]) >np.percentile(speed,65):
            events_ix.append(i)

    return events_ix
    
def get_quiet_events(events,speed):
    speed = abs(speed)
    onsets = [on for on in events["onsets"]] ###get pupil onsets ix
    offsets = [off for off in events["offsets"]] ###get pupil offsets ix
    events_ix = []
    for i,(on,off) in enumerate(zip(onsets,offsets)):
        if np.mean(speed[on:off]) <np.percentile(speed,45):
            events_ix.append(i)
     
    return events_ix


In [ ]:
###aligned trace all trials

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
      
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        onsets_ix = events["onsets"]
  
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore(np.array(df["Running Speed"]))
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

        dil_matrix,dil_times = trial_align(onsets,time,pupil,fps = 20,pre = 10, post = 10 )
        trial_matrix,trial_times = trial_align(onsets,time,dff,fps = 20,pre = 10, post = 10)
        face_matrix,face_times = trial_align(onsets,time,face,fps = 20,pre = 10, post = 10)
        speed_matrix,speed_times = trial_align(onsets,time,speed,fps = 20,pre = 10, post = 10)

        for j,trial in enumerate(dil_matrix):
            baseline = np.mean(pupil[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline

            for i,Time in enumerate(dil_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Pupil","ID" : id, "Date": date}
                df_list.append(new_dict)

        for j,trial in enumerate(trial_matrix):
            baseline = np.mean(dff[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
         
            for i,Time in enumerate(trial_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "dff","ID" : id, "Date": date}
                df_list.append(new_dict)

        for j,trial in enumerate(face_matrix):
            baseline = np.mean(face[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
          
            for i,Time in enumerate(face_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Face","ID" : id, "Date": date}
                df_list.append(new_dict)

        for j,trial in enumerate(speed_matrix):
            baseline = np.mean(speed[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
            for i,Time in enumerate(speed_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Speed","ID" : id, "Date": date}
                df_list.append(new_dict)

align_df = pd.DataFrame.from_dict(df_list)


In [ ]:
plt.figure(figsize = (1,1.2))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type"]).mean().reset_index(), x = "Time from Onset (s)", y = "value", hue = "type",errorbar = "se",palette= ["purple","dodgerblue","darkorange","green"],legend= None,linewidth = 1)
format_ax(ax,(-2,3),(-0.25,1),1,0.25)
ax.set_xlabel("Time from \nDilation Onset (s)")
ax.set_ylabel('Z-Score')
ax.axvline(0,linestyle = "dashed", color = "black",alpha = 0.3)


plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_avgdff_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")




In [ ]:
ax = sns.relplot(data = align_df.query("type != 'Speed'"), x = "Time from Onset (s)", y = "value", hue = "type",errorbar = "se",legend= True,linewidth = 1,kind = "line",col = "ID",col_wrap = 3)
ax.axes[0].set_xlim(-2,3)

In [ ]:
###zoomed in average trace
plt.figure(figsize = (0.7,1.2))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type"]).mean().reset_index().query("type != 'Speed'"), x = "Time from Onset (s)", y = "value", hue = "type",errorbar = "se",palette= ["purple","dodgerblue","green"],legend= None,linewidth = 1)
format_ax(ax,(-0.5,0.5),(-0.25,1),0.5,0.25)
ax.set_xlabel("Time from \ndilation onset (s)")
ax.set_ylabel('Z-Score')

ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_avgdffzoom_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")




In [ ]:
###correlation with dff

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}
df_list = []


for id,dates in id_date.items():
    for date in dates:
      face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
      path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
      df = pd.read_csv(path)
      pupil = np.array(df["Pupil Size"])
      time = np.array(df["Time"])
      speed =zscore(np.array(df["Running Speed"]))
      dff = np.array(df["dFF"])
      face = np.array(pd.read_csv(face_path)["Facial Movement"])
      df["Facial Movement"] = face

      new_dict = {"R":pearsonr(dff,pupil)[0], "type": "P-D", "ID" : id, "Date": date}
      df_list.append(new_dict)
      # new_dict = {"R":pearsonr(dff,speed)[0], "type": "D-S", "ID" : id, "Date": date}
      # df_list.append(new_dict)
      new_dict = {"R":pearsonr(dff,face)[0], "type": "F-D", "ID" : id, "Date": date}
      df_list.append(new_dict)

r_df = pd.DataFrame.from_dict(df_list)

plt.figure(figsize = (0.5,1.2))
g = sns.barplot(data = r_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "R",errorbar="se",fill = None,errwidth= 1)
sns.stripplot(data = r_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "R",color = "black",alpha = 0.3,s = 4,jitter = 0.1)

g.set_yticks(np.arange(-0.2,0.8,0.2))
g.set(ylim=(-0.2, 0.6))
g.xaxis.label.set_visible(False)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_dffcorr_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
### do lag analysis based on pupil-onset aligned traces
pre = 2
post =2


id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
graph_df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        amplitudes = events["amplitudes"]

        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = np.array(df["Running Speed"])
        speed = zscore(speed)
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

        dil_matrix,dil_times = trial_align(onsets,time,pupil,fps = 20,pre = pre, post = post)
        face_matrix,face_times = trial_align(onsets,time,face,fps = 20,pre = pre, post = post)
        wheel_matrix,wheel_times = trial_align(onsets,time,speed,fps = 20,pre = pre, post = post)
        dff_matrix,dff_times = trial_align(onsets,time,dff,fps = 20,pre = pre, post = post)


        corr = correlate(np.mean(dil_matrix,axis=0), np.mean(dff_matrix,axis = 0))
        lags = correlation_lags(len(np.mean(dil_matrix,axis=0)), len(np.mean(dff_matrix,axis = 0)))
        lags = lags/20
        corr /= np.max(corr)
        i = np.argmax(corr)
        delay = lags[i]
        lag_dict = {"Cross-Correlation Delay (s)" : delay,"ID": id, "date" : date,"type":"P-D"}
        df_list.append(lag_dict)

        for lag,c in zip(lags,corr):
            graph_dict = {"Lags (s)": lag,"Cross-Correlation": c,"ID": id, "date" : date,"type":"P-D"}
            graph_df_list.append(graph_dict)
        

        corr = correlate(np.mean(face_matrix,axis=0), np.mean(dff_matrix,axis = 0))
        lags = correlation_lags(len(np.mean(face_matrix,axis=0)), len(np.mean(dff_matrix,axis = 0)))
        lags = lags/20
        corr /= np.max(corr)
        i = np.argmax(corr)
        delay = lags[i]
        lag_dict = {"Cross-Correlation Delay (s)" : delay,"ID": id, "date" : date,"type":"F-D"}
        df_list.append(lag_dict)

        for lag,c in zip(lags,corr):
            graph_dict = {"Lags (s)" : lag,"Cross-Correlation": c,"ID": id, "date" : date,"type":"F-D"}
            graph_df_list.append(graph_dict)
        
        corr = correlate(np.mean(wheel_matrix,axis=0), np.mean(dff_matrix,axis = 0))
        lags = correlation_lags(len(np.mean(wheel_matrix,axis=0)), len(np.mean(dff_matrix,axis = 0)))
        lags = lags/20
        corr /= np.max(corr)
        i = np.argmax(corr)
        delay = lags[i]
        lag_dict = {"Cross-Correlation Delay (s)" : delay,"ID": id, "date" : date,"type":"S-D"}
        #df_list.append(lag_dict)

        for lag,c in zip(lags,corr):
            graph_dict = {"Lags (s)" : lag,"Cross-Correlation": c,"ID": id, "date" : date,"type":"S-D"}
            graph_df_list.append(graph_dict)


lag_df = pd.DataFrame.from_dict(df_list)
graph_df = pd.DataFrame.from_dict(graph_df_list)

###generate delay bar graph
plt.figure(figsize = (0.5,1.2))
g = sns.barplot(data = lag_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Cross-Correlation Delay (s)",errorbar= "se",fill = None,errwidth= 1)
sns.stripplot(data = lag_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Cross-Correlation Delay (s)",color = "black",alpha = 0.3,s = 4,jitter = 0.1)


g.set_yticks(np.arange(-0.5,2.5,0.5))
g.set(ylim=(-0.5, 2))
g.xaxis.label.set_visible(False)
g.set_ylabel("Cross-correlation\ndelay (s)")

#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_dfflag_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")





In [ ]:
lag_df.query("type == 'F-D'")

In [ ]:
###cross correlation line graph

plt.figure(figsize = (0.7,1.2))
ax = sns.lineplot(data = graph_df.groupby(["ID",'type','Lags (s)']).mean().reset_index().query("type != 'S-D'"), x = "Lags (s)", y = "Cross-Correlation",errorbar="se",hue = "type",palette = ["purple","dodgerblue","darkorange"],legend=None,linewidth = 1)
format_ax(ax,(-3,3),(-2.5,1.5),3,1)
ax.set_ylabel("Normalized\ncross-correlation (s)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)


plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_lag_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###align to pupil dilation onset binned by quartile


id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        onsets_ix = events["onsets"]
        amplitudes = events["amplitudes"]
        binned_amps = np.array(pd.qcut(amplitudes,q = 4,labels = [1,2,3,4]))
   
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = filter_pupil(np.array(df["Pupil Size"]))
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

       

        dil_matrix,dil_times = trial_align(onsets,time,pupil,fps = 20,pre = 10, post = 10 )

        for j,trial in enumerate(dil_matrix):
            baseline = np.mean(pupil[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
            for i,Time in enumerate(dil_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Pupil","ID" : id, "Date": date,"amp":amplitudes[j],"animal_amp_bin":binned_amps[j]}
                df_list.append(new_dict)

        trial_matrix,trial_times = trial_align(onsets,time,dff,fps = 20,pre = 10, post = 10)
        for j,trial in enumerate(trial_matrix):
            baseline = np.mean(dff[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
            for i,Time in enumerate(trial_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "dff","ID" : id, "Date": date,"amp":amplitudes[j],"animal_amp_bin":binned_amps[j]}
                df_list.append(new_dict)
        
        face_matrix,face_times = trial_align(onsets,time,face,fps = 20,pre = 10, post = 10)
        for j,trial in enumerate(face_matrix):
            baseline = np.mean(face[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
            for i,Time in enumerate(face_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "face","ID" : id, "Date": date,"amp":amplitudes[j],"animal_amp_bin":binned_amps[j]}
                df_list.append(new_dict)

align_df = pd.DataFrame.from_dict(df_list)

In [ ]:
###plot different quartiles of pupil aligned to pupil onset
plt.figure(figsize = (3.25,3))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","animal_amp_bin"]).mean().reset_index().query("type == 'dff'"), x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "animal_amp_bin",legend = None,linewidth = 1)
#ax.set_xlabel("Time from \nDilation Onset (s)")
ax.xaxis.label.set_visible(False)

ax.set_ylabel("Pupil Size \n(z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-2,3.5),(-0.5,3),0.5,0.5)



In [ ]:
###plot different quartiles of pupil aligned to pupil onset
plt.figure(figsize = (5,5))
ax = sns.lineplot(data = align_df.query("animal_amp_bin == 4"), x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "type",legend = True,linewidth = 1)
#ax.set_xlabel("Time from \nDilation Onset (s)")
ax.xaxis.label.set_visible(False)

ax.set_ylabel("Pupil Size \n(z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-0.5,8.5),(-0.5,3),0.5,0.5)
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_pupil_quartile_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###plot different quartiles of pupil aligned to pupil onset
plt.figure(figsize = (1.25,1))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","animal_amp_bin"]).mean().reset_index().query("type == 'Pupil'").query("animal_amp_bin == [1,4]"), x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "animal_amp_bin",palette= ["cornflowerblue","darkblue"],legend = None,linewidth = 1)
#ax.set_xlabel("Time from \nDilation Onset (s)")
ax.xaxis.label.set_visible(False)

ax.set_ylabel("Pupil Size \n(z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-0.5,0.5),(-0.5,3),0.5,0.5)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_pupil_quartile_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###plot different quartiles of pupil aligned to pupil onset
plt.figure(figsize = (1.25,1.25))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","animal_amp_bin"]).mean().reset_index().query("type == 'dff'").query("animal_amp_bin == [1,4]"), x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "animal_amp_bin",palette= ["palegreen","darkgreen"],legend = None,linewidth = 1)
ax.set_xlabel("Time from \nDilation Onset (s)")
ax.set_ylabel("∆ F/F (z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-0.5,0.5),(-0.5,2),0.5,0.5)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_dff_quartile_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###align to pupil dilation peak binned by quartile


id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        peaks = [on/20 for on in events["peak_ix"]] ### get pupil peaks
        onsets_ix = events["onsets"]
        amplitudes = events["amplitudes"]
        binned_amps = np.array(pd.qcut(amplitudes,q = 4,labels = [1,2,3,4]))
   
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = filter_pupil(np.array(df["Pupil Size"]))
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

       

        dil_matrix,dil_times = trial_align(peaks,time,pupil,fps = 20,pre = 10, post = 10 )

        for j,trial in enumerate(dil_matrix):
            baseline = np.mean(pupil[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
            for i,Time in enumerate(dil_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Pupil","ID" : id, "Date": date,"amp":amplitudes[j],"animal_amp_bin":binned_amps[j]}
                df_list.append(new_dict)

        trial_matrix,trial_times = trial_align(peaks,time,dff,fps = 20,pre = 10, post = 10)
        for j,trial in enumerate(trial_matrix):
            baseline = np.mean(dff[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
            for i,Time in enumerate(trial_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "dff","ID" : id, "Date": date,"amp":amplitudes[j],"animal_amp_bin":binned_amps[j]}
                df_list.append(new_dict)

align_df = pd.DataFrame.from_dict(df_list)

In [ ]:
###plot different quartiles of pupil aligned to pupil peak
plt.figure(figsize = (1.25,1))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","animal_amp_bin"]).mean().reset_index().query("type == 'Pupil'").query("animal_amp_bin == [1,4]"), x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "animal_amp_bin",palette= ["cornflowerblue","darkblue"],legend = None,linewidth = 1)
#ax.set_xlabel("Time from \nDilation Peak (s)")
ax.xaxis.label.set_visible(False)

ax.set_ylabel("Pupil Size \n(z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-0.5,1),(-0.5,3),0.5,0.5)
ax.spines[['left']].set_visible(False)
ax.get_yaxis().set_visible(False)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_pupil_peakquartile_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###plot different quartiles of pupil aligned to pupil peak
plt.figure(figsize = (1.25,1.25))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","animal_amp_bin"]).mean().reset_index().query("type == 'dff'").query("animal_amp_bin == [1,4]"), x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "animal_amp_bin",palette= ["palegreen","darkgreen"],legend = None,linewidth = 1)
ax.set_xlabel("Time from \nDilation Peak (s)")
ax.set_ylabel("∆ F/F (z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-0.5,1),(-0.5,2),0.5,0.5)
ax.spines[['left']].set_visible(False)
ax.get_yaxis().set_visible(False)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_dff_peakquartile_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
### mean dff within amplitude quartiles

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}
labels = ["1","2","3","4"]
df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        peak_ix = events["peak_ix"]
        onsets = [on for on in events["onsets"]] ###get pupil onsets ix
        onsets_ix = events["onsets"]
        offsets = [off for off in events["offsets"]] ###get pupil offsets ix
        amplitudes = events["amplitudes"]
        binned_amps = np.array(pd.qcut(amplitudes,q = 4,labels = labels))
     

  
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

       

     
        for i,(on,off) in enumerate(zip(onsets,offsets)):
          baseline = np.mean(dff[on - 40:on - 10])
          pre_dff =np.mean(dff[on-5:on]) - baseline
          during_dff = np.mean(dff[on:off]) - baseline


          new_dict = { "trial" : i,"ID" : id, "Date": date,"amp":amplitudes[i],"animal_amp_bin":binned_amps[i],"Pre Onset ∆ F/F":pre_dff,"∆ F/F": during_dff}
          df_list.append(new_dict)


bar_df = pd.DataFrame.from_dict(df_list)


In [ ]:
plt.figure(figsize = (1,1.25))
g = sns.barplot(data = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index(), x = "animal_amp_bin", y = "Pre Onset ∆ F/F",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index(), x = "animal_amp_bin", y = "Pre Onset ∆ F/F",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Dilation Amplitude\n Quartile")
g.set_yticks(np.arange(-0.25,1,0.25))
g.set(ylim=(-0.25,0.75))
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_ampbin_predff_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index()

In [ ]:
###stats
q1 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '1'")["Pre Onset ∆ F/F"]
q2 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '2'")["Pre Onset ∆ F/F"]
q3 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '3'")["Pre Onset ∆ F/F"]
q4 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '4'")["Pre Onset ∆ F/F"]
kruskal(q1,q2,q3,q4)

In [ ]:
plt.figure(figsize = (1,1.25))
g = sns.barplot(data = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index(), x = "animal_amp_bin", y = "∆ F/F",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index(), x = "animal_amp_bin", y = "∆ F/F",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Dilation Amplitude\n Quartile")
g.set_yticks(np.arange(0,2,0.5))
g.set(ylim=(0,1))
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_ampbin_dff_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
q1 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '1'")["∆ F/F"]
q2 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '2'")["∆ F/F"]
q3 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '3'")["∆ F/F"]
q4 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '4'")["∆ F/F"]
kruskal(q1,q2,q3,q4)

In [ ]:
###align to pupil dilation onset binned by face quartile


id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}
labels = ["1","2"]

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        onsets_ix = events["onsets"]
        amplitudes = events["amplitudes"]
   
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

       
        face_deltas = []
        face_matrix,face_times = trial_align(onsets,time,face,fps = 20,pre = 10, post = 10 )
        for j,trial in enumerate(face_matrix):
            pre_face = np.mean(trial[160:190])
            post_face = np.mean(trial[200:300])
            face_deltas.append(post_face-pre_face)
        
        binned_face = np.array(pd.qcut(face_deltas,q = 2,labels = labels))

        
    

        for j,trial in enumerate(face_matrix):
            baseline = np.mean(face[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
            for i,Time in enumerate(face_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Face","ID" : id, "Date": date,"amp":amplitudes[j],"animal_face_bin":binned_face[j]}
                df_list.append(new_dict)

        trial_matrix,trial_times = trial_align(onsets,time,dff,fps = 20,pre = 10, post = 10)
        for j,trial in enumerate(trial_matrix):
            baseline = np.mean(dff[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
            for i,Time in enumerate(trial_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "dff","ID" : id, "Date": date,"amp":amplitudes[j],"animal_face_bin":binned_face[j]}
                df_list.append(new_dict)

align_df = pd.DataFrame.from_dict(df_list)

In [ ]:
###plot different quartiles of pupil aligned to pupil onset
plt.figure(figsize = (1.35,1.5))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","animal_face_bin"]).mean().reset_index().query("type == 'Face'"), x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "animal_face_bin",palette= ["violet","purple"],legend = None,linewidth = 1)
ax.set_xlabel("Time from \ndilation onset (s)")

ax.set_ylabel("Facial\nmovement (z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-1,3),(-0.5,1),1,0.5)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/S4_face_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###plot different quartiles of pupil aligned to pupil onset
plt.figure(figsize = (1.35,1.5))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","animal_face_bin"]).mean().reset_index().query("type == 'dff'"), x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "animal_face_bin",palette= ["palegreen","darkgreen"],legend = None,linewidth = 1)
ax.set_xlabel("Time from \nDilation Onset (s)")

ax.set_ylabel("∆ F/F (z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-1,3),(-0.5,1),1,0.5)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/S4_dff_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
### mean dff within amplitude quartiles

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}
labels = ["1","2"]
df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        peak_ix = events["peak_ix"]
        onsets = [on for on in events["onsets"]] ###get pupil onsets ix
        onsets_times = [on/20 for on in events["onsets"]]
        offsets = [off for off in events["offsets"]] ###get pupil offsets ix
        amplitudes = events["amplitudes"]
     

  
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])


        face_deltas = []
        face_matrix,face_times = trial_align(onsets_times,time,face,fps = 20,pre = 10, post = 10 )
        for j,trial in enumerate(face_matrix):
          pre_face = np.mean(trial[160:190])
          post_face = np.mean(trial[200:300])
          face_deltas.append(post_face-pre_face)
        
        binned_face = np.array(pd.qcut(face_deltas,q = 2,labels = labels))

       

     
        for i,(on,off) in enumerate(zip(onsets,offsets)):
          baseline = np.mean(dff[on - 40:on - 10])
          pre_dff =np.mean(dff[on-5:on]) - baseline
          during_dff = np.mean(dff[on:off]) - baseline


          new_dict = { "trial" : i,"ID" : id, "Date": date,"amp":amplitudes[i],"animal_face_bin":binned_face[i],"Pre Onset ∆ F/F":pre_dff,"∆ F/F": during_dff}
          df_list.append(new_dict)

bar_df = pd.DataFrame.from_dict(df_list)


In [ ]:
plt.figure(figsize = (1.35,1.5))
g = sns.barplot(data = bar_df.groupby(["ID",'animal_face_bin']).mean().reset_index(), x = "animal_face_bin", y = "∆ F/F",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = bar_df.groupby(["ID",'animal_face_bin']).mean().reset_index(), x = "animal_face_bin", y = "∆ F/F",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Facial Movement\n Quartile")
g.set_yticks(np.arange(0,2,0.5))
g.set(ylim=(0,1))
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/S4_face_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
q1 = bar_df.groupby(["ID",'animal_face_bin']).mean().reset_index().query("animal_face_bin == '1'")["∆ F/F"]
q2 = bar_df.groupby(["ID",'animal_face_bin']).mean().reset_index().query("animal_face_bin == '2'")["∆ F/F"]
q3 = bar_df.groupby(["ID",'animal_face_bin']).mean().reset_index().query("animal_face_bin == '3'")["∆ F/F"]
q4 = bar_df.groupby(["ID",'animal_face_bin']).mean().reset_index().query("animal_face_bin == '4'")["∆ F/F"]
kruskal(q1,q2,q3,q4)

In [ ]:
###align to pupil dilation onset binned by duration


id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        onsets_ix = events["onsets"]
        durations = events["durations"]
        binned_durs = np.array(pd.qcut(durations,q = 4,labels = [1,2,3,4]))
   
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

       

        dil_matrix,dil_times = trial_align(onsets,time,pupil,fps = 20,pre = 10, post = 10 )

        for j,trial in enumerate(dil_matrix):
            baseline = np.mean(pupil[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
            for i,Time in enumerate(dil_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Pupil","ID" : id, "Date": date,"dur":durations[j],"animal_dur_bin":binned_durs[j]}
                df_list.append(new_dict)

        trial_matrix,trial_times = trial_align(onsets,time,dff,fps = 20,pre = 10, post = 10)
        for j,trial in enumerate(trial_matrix):
            baseline = np.mean(dff[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
            for i,Time in enumerate(trial_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "dff","ID" : id, "Date": date,"dur":durations[j],"animal_dur_bin":binned_durs[j]}
                df_list.append(new_dict)

align_df = pd.DataFrame.from_dict(df_list)

In [ ]:
###plot different quartiles of pupil aligned to pupil onset
plt.figure(figsize = (1,0.75))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","animal_dur_bin"]).mean().reset_index().query("type == 'Pupil'").query("animal_dur_bin == [1,4]"), x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "animal_dur_bin",palette= ["cornflowerblue","darkblue"],legend = None,linewidth = 1)
#ax.set_xlabel("Time from \nDilation Onset (s)")
ax.xaxis.label.set_visible(False)

ax.set_ylabel("Pupil Size \n(z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-0.5,1),(-0.5,1),0.5,0.5)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_pupil_durquartile_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###plot different quartiles of pupil aligned to pupil onset
plt.figure(figsize = (1,1.5))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","animal_dur_bin"]).mean().reset_index().query("type == 'dff'").query("animal_dur_bin == [1,4]"), x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "animal_dur_bin",palette= ["palegreen","darkgreen"],legend = None,linewidth = 1)
ax.set_xlabel("Time from \nDilation Onset (s)")
ax.set_ylabel("∆ F/F (z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-0.5,1),(-0.5,1.5),0.5,0.5)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_dff_durquartile_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###align to pupil dilation onset binned by duration


id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        peaks = [on/20 for on in events["peak_ix"]] ### get pupil peaks
        onsets_ix = events["onsets"]
        durations = events["durations"]
        binned_durs = np.array(pd.qcut(durations,q = 4,labels = [1,2,3,4]))
   
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

       

        dil_matrix,dil_times = trial_align(peaks,time,pupil,fps = 20,pre = 10, post = 10 )

        for j,trial in enumerate(dil_matrix):
            baseline = np.mean(pupil[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
            for i,Time in enumerate(dil_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Pupil","ID" : id, "Date": date,"dur":durations[j],"animal_dur_bin":binned_durs[j]}
                df_list.append(new_dict)

        trial_matrix,trial_times = trial_align(peaks,time,dff,fps = 20,pre = 10, post = 10)
        for j,trial in enumerate(trial_matrix):
            baseline = np.mean(dff[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
            for i,Time in enumerate(trial_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "dff","ID" : id, "Date": date,"dur":durations[j],"animal_dur_bin":binned_durs[j]}
                df_list.append(new_dict)

align_df = pd.DataFrame.from_dict(df_list)

In [ ]:
###plot different quartiles of pupil aligned to pupil peak
plt.figure(figsize = (1,0.75))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","animal_dur_bin"]).mean().reset_index().query("type == 'Pupil'").query("animal_dur_bin == [1,4]"), x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "animal_dur_bin",palette= ["cornflowerblue","darkblue"],legend = None,linewidth = 1)
#ax.set_xlabel("Time from \nDilation Peak (s)")
ax.xaxis.label.set_visible(False)

ax.set_ylabel("Pupil Size \n(z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-0.5,1),(-0.5,3),0.5,0.5)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_pupil_peakdurquartile_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###plot different quartiles of pupil aligned to pupil peak
plt.figure(figsize = (1,1.5))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","animal_dur_bin"]).mean().reset_index().query("type == 'dff'").query("animal_dur_bin == [1,4]"), x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "animal_dur_bin",palette= ["palegreen","darkgreen"],legend = None,linewidth = 1)
ax.set_xlabel("Time from \nDilation Peak (s)")
ax.set_ylabel("∆ F/F (z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-0.5,1),(-0.5,3),0.5,0.5)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_dff_peakdurquartile_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
### mean dff within amplitude quartiles

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}
labels = ["1","2","3","4"]
df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        peak_ix = events["peak_ix"]
        onsets = [on for on in events["onsets"]] ###get pupil onsets ix
        onsets_ix = events["onsets"]
        offsets = [off for off in events["offsets"]] ###get pupil offsets ix
        durations = events["durations"]
        binned_durs = np.array(pd.qcut(durations,q = 4,labels = labels))
     

  
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

       

     
        for i,(on,off) in enumerate(zip(onsets,offsets)):
          baseline = np.mean(dff[on - 40:on - 10])
          pre_dff =np.mean(dff[on-5:on]) - baseline
          during_dff = np.mean(dff[on:peak_ix[i]]) - baseline


          new_dict = { "trial" : i,"ID" : id, "Date": date,"dur":durations[i],"animal_dur_bin":binned_durs[i],"Pre Onset ∆ F/F":pre_dff,"∆ F/F": during_dff}
          df_list.append(new_dict)


bar_df = pd.DataFrame.from_dict(df_list)


In [ ]:
plt.figure(figsize = (1,1.5))
g = sns.barplot(data = bar_df.groupby(["ID",'animal_dur_bin']).mean().reset_index(), x = "animal_dur_bin", y = "Pre Onset ∆ F/F",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = bar_df.groupby(["ID",'animal_dur_bin']).mean().reset_index(), x = "animal_dur_bin", y = "Pre Onset ∆ F/F",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Dilation Duration\n Quartile")
g.set_yticks(np.arange(-0.25,1,0.25))
g.set(ylim=(-0.25,0.75))
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_durbin_predff_durbar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
q1 = bar_df.groupby(["ID",'animal_dur_bin']).mean().reset_index().query("animal_dur_bin == '1'")["Pre Onset ∆ F/F"]
q2 = bar_df.groupby(["ID",'animal_dur_bin']).mean().reset_index().query("animal_dur_bin == '2'")["Pre Onset ∆ F/F"]
q3 = bar_df.groupby(["ID",'animal_dur_bin']).mean().reset_index().query("animal_dur_bin == '3'")["Pre Onset ∆ F/F"]
q4 = bar_df.groupby(["ID",'animal_dur_bin']).mean().reset_index().query("animal_dur_bin == '4'")["Pre Onset ∆ F/F"]
kruskal(q1,q2,q3,q4)

In [ ]:
plt.figure(figsize = (1,1.5))
g = sns.barplot(data = bar_df.groupby(["ID",'animal_dur_bin']).mean().reset_index(), x = "animal_dur_bin", y = "∆ F/F",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = bar_df.groupby(["ID",'animal_dur_bin']).mean().reset_index(), x = "animal_dur_bin", y = "∆ F/F",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Dilation Duration\n Quartile")
g.set_yticks(np.arange(0,2,0.5))
g.set(ylim=(0,1.5))
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_ampbin_dff_durbar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
q1 = bar_df.groupby(["ID",'animal_dur_bin']).mean().reset_index().query("animal_dur_bin == '1'")["∆ F/F"]
q2 = bar_df.groupby(["ID",'animal_dur_bin']).mean().reset_index().query("animal_dur_bin == '2'")["∆ F/F"]
q3 = bar_df.groupby(["ID",'animal_dur_bin']).mean().reset_index().query("animal_dur_bin == '3'")["∆ F/F"]
q4 = bar_df.groupby(["ID",'animal_dur_bin']).mean().reset_index().query("animal_dur_bin == '4'")["∆ F/F"]
kruskal(q1,q2,q3,q4)

In [ ]:
###generate movement vs non movement aligned traces

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
"004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
    
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
  
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        raw_speed = np.array(df["Running Speed"])
     
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

        move_events = get_movement_events(events,raw_speed)
        quiet_events = get_quiet_events(events,raw_speed)

        dil_matrix,dil_times = trial_align(onsets,time,pupil,fps = 20,pre = 10, post = 10 )

        for j,trial in enumerate(dil_matrix):
            baseline = np.mean(trial[170:190])
            trial = trial-baseline
            if j in quiet_events:
                type = "quiet"
            elif j in move_events:
                type = "running"
            else:
                type = "neither"
            for i,Time in enumerate(dil_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Pupil","ID" : id, "Date": date,"Speed":type}
                df_list.append(new_dict)
                
        dff_matrix,dff_times = trial_align(onsets,time,dff,fps = 20,pre = 10, post = 10)
        for j,trial in enumerate(dff_matrix):
            baseline = np.mean(trial[170:190])
            trial = trial-baseline
            if j in quiet_events:
                type = "quiet"
            elif j in move_events:
                type = "running"
            else:
                type = "neither"
            for i,Time in enumerate(dff_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "dff","ID" : id, "Date": date,"Speed":type}
                df_list.append(new_dict)

        speed_matrix,speed_times = trial_align(onsets,time,speed,fps = 20,pre = 10, post = 10)

        for j,trial in enumerate(speed_matrix):
            baseline = np.mean(trial[170:190])
            trial = trial-baseline
            if j in move_events:
                type = "running"
            elif j in quiet_events:
                type = "quiet"
            else:
                type = "neither"
            for i,Time in enumerate(speed_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Speed","ID" : id, "Date": date,"Speed":type}
                df_list.append(new_dict)

align_df = pd.DataFrame.from_dict(df_list)
align_df = align_df.query("Speed != 'neither'")

In [ ]:
plt.figure(figsize = (1.35,1.5))

ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","Speed"]).mean().reset_index().query("Speed == 'running'").query("type != 'Speed'"), x = "Time from Onset (s)", y = "value", hue = "type",errorbar = "se",legend = None,linewidth = 1, palette= ["dodgerblue","green"])
format_ax(ax,(-1,5),(-0.5,2),1,0.5)

ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)

ax.set_xlabel("Time from \nDilation Onset (s)")
ax.set_ylabel('Z-Score')
  
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/S4_ running_trace_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")

In [ ]:
plt.figure(figsize = (1.35,1.5))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","Speed"]).mean().reset_index().query("Speed == 'quiet'").query("type != 'Speed'"), x = "Time from Onset (s)", y = "value", hue = "type",errorbar = "se",legend = None,linewidth = 1, palette= ["dodgerblue","green"])
format_ax(ax,(-1,5),(-0.5,2),1,0.5)

ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)

ax.set_xlabel("Time from \nDilation Onset (s)")
ax.set_ylabel('Z-Score')

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/S4_ quiet_trace_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")

In [ ]:
###running vs quiet pupil dilations

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        offsets = [off/20 for off in events["offsets"]] ###get pupil offsets
        amplitudes = events["amplitudes"]
        peak_ix = events["peak_ix"]



        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        raw_speed = np.array(df["Running Speed"])

        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

        move_events = get_movement_events(events,raw_speed)
        quiet_events = get_quiet_events(events,raw_speed)

        for event in list(range(0,len(onsets))):
            on = np.array(events["onsets"])[event]
            off = np.array(events["offsets"])[event]
        
            baseline = np.mean(dff[on - 40:on - 10])
            pre_dff = np.mean(dff[on-5:on]) - baseline
            during_dff = np.mean(dff[on:off]) - baseline

            if event in move_events:
                type = "Running"
            elif event in quiet_events:
                type = "Quiet"
            else:
                type = "neither"
            
            new_dict = { "trial" : event,"ID" : id, "Date": date,"type":type,"Pre Onset ∆ F/F":pre_dff,"∆ F/F": during_dff}

            df_list.append(new_dict)

df = pd.DataFrame.from_dict(df_list)
df = df.query("type != 'neither'")


In [ ]:
df.query("type == 'Quiet'")["ID"].value_counts()

In [ ]:
df.query("type == 'Running'")["ID"].value_counts()

In [ ]:
plt.figure(figsize = (1.35,1.5))
g = sns.barplot(data = df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Pre Onset ∆ F/F",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Pre Onset ∆ F/F",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
g.xaxis.label.set_visible(False)

g.set_yticks(np.arange(-0.25,1,0.25))
g.set(ylim=(-0.25,0.75))
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/S4_ pre_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")

In [ ]:
plt.figure(figsize = (1.35,1.5))
g = sns.barplot(data = df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "∆ F/F",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "∆ F/F",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
g.xaxis.label.set_visible(False)

g.set_yticks(np.arange(0,1,0.25))
g.set(ylim=(0,0.75))
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/S4_ during_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")

In [ ]:
###stats
stat_df = df.groupby(["ID",'type']).mean().reset_index()
group1 = stat_df[stat_df['type']=="Quiet"]
group2 = stat_df[stat_df['type']=="Running"]
wilcoxon(group1["∆ F/F"], group2["∆ F/F"],nan_policy= "omit")

In [ ]:
###stratify within quiet dilations

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        offsets = [off/20 for off in events["offsets"]] ###get pupil offsets
        amplitudes = events["amplitudes"]
        peak_ix = events["peak_ix"]



        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        raw_speed = np.array(df["Running Speed"])

        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

        move_events = get_movement_events(events,raw_speed)
        quiet_events = get_quiet_events(events,raw_speed)
        quiet_amps = []
        for i,amp in enumerate(amplitudes):
            if i in quiet_events:
                quiet_amps.append(amp)
        binned_amps = np.array(pd.qcut(quiet_amps,q = 2 ,labels = ["1","2"]))



        
        for i,event in enumerate(quiet_events):
            on = np.array(events["onsets"])[event]
            off = np.array(events["offsets"])[event]
            baseline = np.mean(dff[on - 40:on - 10])
            pre_dff = np.mean(dff[on-5:on]) - baseline
            during_dff = np.mean(dff[on:off]) - baseline
            during_speed = np.mean(raw_speed[on:off])
            new_dict = { "trial" : event,"ID" : id, "Date": date,"Pre Onset ∆ F/F":pre_dff,"∆ F/F": during_dff,"amp_bin":binned_amps[i],"speed":during_speed,"amplitude":quiet_amps[i]}
            df_list.append(new_dict)

 

df = pd.DataFrame.from_dict(df_list)


In [ ]:
df["ID"].value_counts()

In [ ]:
plt.figure(figsize = (1,1.5))
g = sns.barplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "amplitude",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "amplitude",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Pupil Amplitude\n Quartile")
# g.set_yticks(np.arange(0,2,0.5))
# g.set(ylim=(0,1))
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_facebin_dff_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
plt.figure(figsize = (0.5,1.25))
g = sns.barplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "∆ F/F",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "∆ F/F",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Pupil Amplitude\n Median")
g.set_yticks(np.arange(0,0.75,0.25))
g.set(ylim=(0,0.5))
g.xaxis.label.set_visible(False)
labels = ["Low","High"]
g.set_xticklabels(labels)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_facebin_dff_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = df.groupby(["ID",'amp_bin']).mean().reset_index()
group1 = stat_df[stat_df['amp_bin']=="1"]
group2 = stat_df[stat_df['amp_bin']=="2"]
wilcoxon(group1["∆ F/F"], group2["∆ F/F"],nan_policy= "omit")

In [ ]:
plt.figure(figsize = (0.5,1.25))
g = sns.barplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "speed",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "speed",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Pupil Amplitude\n Quartile")
g.set_ylabel("Speed (AU)")

g.set_yticks(np.arange(0,1.25,0.25))
g.set(ylim=(0,1))
g.xaxis.label.set_visible(False)
labels = ["Low","High"]
g.set_xticklabels(labels)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_quietbin_speed_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = df.groupby(["ID",'amp_bin']).mean().reset_index()
group1 = stat_df[stat_df['amp_bin']=="1"]
group2 = stat_df[stat_df['amp_bin']=="2"]
wilcoxon(group1["speed"], group2["speed"],nan_policy= "omit")

In [ ]:
###stratify within run dilations

id_date = {
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        offsets = [off/20 for off in events["offsets"]] ###get pupil offsets
        amplitudes = events["amplitudes"]
        peak_ix = events["peak_ix"]



        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        raw_speed = np.array(df["Running Speed"])

        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

        move_events = get_movement_events(events,raw_speed)
        quiet_events = get_quiet_events(events,raw_speed)
        move_amps = []
        for i,amp in enumerate(amplitudes):
            if i in move_events:
                move_amps.append(amp)
        binned_amps = np.array(pd.qcut(move_amps,q = 2 ,labels = ["1","2"]))



        
        for i,event in enumerate(move_events):
            on = np.array(events["onsets"])[event]
            off = np.array(events["offsets"])[event]
            baseline = np.mean(dff[on - 40:on - 10])
            pre_dff = np.mean(dff[on-5:on]) - baseline
            during_dff = np.mean(dff[on:off]) - baseline
            during_speed = np.mean(raw_speed[on:off])
            new_dict = { "trial" : event,"ID" : id, "Date": date,"Pre Onset ∆ F/F":pre_dff,"∆ F/F": during_dff,"amp_bin":binned_amps[i],"speed":during_speed,"amplitude":move_amps[i]}
            df_list.append(new_dict)

 

df = pd.DataFrame.from_dict(df_list)


In [ ]:
df["ID"].value_counts()

In [ ]:
plt.figure(figsize = (0.5,1))
g = sns.barplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "amplitude",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "amplitude",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Pupil Amplitude\n Quartile")
# g.set_yticks(np.arange(0,2,0.5))
# g.set(ylim=(0,1))
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_facebin_dff_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
plt.figure(figsize = (0.5,1.25))
g = sns.barplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "∆ F/F",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "∆ F/F",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Pupil Amplitude\n Quartile")
g.set_yticks(np.arange(0,2,0.5))
g.set(ylim=(0,1))
g.xaxis.label.set_visible(False)
labels = ["Low","High"]
g.set_xticklabels(labels)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_runbin_dff_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = df.groupby(["ID",'amp_bin']).mean().reset_index()
group1 = stat_df[stat_df['amp_bin']=="1"]
group2 = stat_df[stat_df['amp_bin']=="2"]
wilcoxon(group1["∆ F/F"], group2["∆ F/F"],nan_policy= "omit")

In [ ]:
###stats
stat_df = df.groupby(["ID",'amp_bin']).mean().reset_index()
group1 = stat_df[stat_df['amp_bin']=="1"]
group2 = stat_df[stat_df['amp_bin']=="2"]
wilcoxon(group1["speed"], group2["speed"],nan_policy= "omit")

In [ ]:
plt.figure(figsize = (0.5,1.25))
g = sns.barplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "speed",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "speed",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Pupil Amplitude\n Quartile")
g.set_ylabel("Speed (AU)")

g.set_yticks(np.arange(0,25,5))
g.set(ylim=(0,20))
g.xaxis.label.set_visible(False)
labels = ["Low","High"]
g.set_xticklabels(labels)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_runbin_speed_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = df.groupby(["ID",'amp_bin']).mean().reset_index()
group1 = stat_df[stat_df['amp_bin']=="1"]
group2 = stat_df[stat_df['amp_bin']=="2"]
wilcoxon(group1["speed"], group2["speed"],nan_policy= "omit")

In [ ]:
###median split quiet traces

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
"004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
    
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        amplitudes = events["amplitudes"]

  
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        raw_speed = np.array(df["Running Speed"])
     
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

        move_events = get_movement_events(events,raw_speed)
        quiet_events = get_quiet_events(events,raw_speed)
        quiet_amps = []
        for i,amp in enumerate(amplitudes):
            if i in quiet_events:
                quiet_amps.append(amp)
        binned_amps = np.array(pd.qcut(quiet_amps,q = 2 ,labels = ["1","2"]))

        quiet_onsets = np.array(onsets)[np.array(quiet_events)]

        dil_matrix,dil_times = trial_align(quiet_onsets,time,pupil,fps = 20,pre = 10, post = 10 )

        for j,trial in enumerate(dil_matrix):
            baseline = np.mean(trial[170:190])
            trial = trial-baseline
            for i,Time in enumerate(dil_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Pupil","ID" : id, "Date": date,"bin":binned_amps[j]}
                df_list.append(new_dict)
                
        dff_matrix,dff_times = trial_align(quiet_onsets,time,dff,fps = 20,pre = 10, post = 10)
        for j,trial in enumerate(dff_matrix):
            baseline = np.mean(trial[170:190])
            trial = trial-baseline
            if j in quiet_events:
                type = "quiet"
            elif j in move_events:
                type = "running"
            else:
                type = "neither"
            for i,Time in enumerate(dff_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "dff","ID" : id, "Date": date,"bin":binned_amps[j]}
                df_list.append(new_dict)

        speed_matrix,speed_times = trial_align(quiet_onsets,time,speed,fps = 20,pre = 10, post = 10)

        for j,trial in enumerate(speed_matrix):
            baseline = np.mean(trial[170:190])
            trial = trial-baseline
            if j in move_events:
                type = "running"
            elif j in quiet_events:
                type = "quiet"
            else:
                type = "neither"
            for i,Time in enumerate(speed_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Speed","ID" : id, "Date": date,"bin":binned_amps[j]}
                df_list.append(new_dict)
        
        face_matrix,face_times = trial_align(quiet_onsets,time,face,fps = 20,pre = 10, post = 10)

        for j,trial in enumerate(face_matrix):
            baseline = np.mean(trial[170:190])
            trial = trial-baseline
            if j in move_events:
                type = "running"
            elif j in quiet_events:
                type = "quiet"
            else:
                type = "neither"
            for i,Time in enumerate(speed_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Face","ID" : id, "Date": date,"bin":binned_amps[j]}
                df_list.append(new_dict)

        

align_df = pd.DataFrame.from_dict(df_list)


In [ ]:
###plot different quartiles of pupil aligned to pupil onset
plt.figure(figsize = (1.9,1))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","bin"]).mean().reset_index().query("type == 'Face'"), x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "bin",palette= ["cornflowerblue","darkblue"],legend = None,linewidth = 1)
#ax.set_xlabel("Time from \nDilation Onset (s)")
ax.xaxis.label.set_visible(False)

ax.set_ylabel("Pupil Size \n(z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-1,3),(-0.2,1),1,0.2)
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_quiet_median_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###plot different quartiles of pupil aligned to pupil onset
plt.figure(figsize = (1.9,1))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","bin"]).mean().reset_index().query("type == 'Pupil'"), x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "bin",palette= ["cornflowerblue","darkblue"],legend = None,linewidth = 1)
#ax.set_xlabel("Time from \nDilation Onset (s)")
ax.xaxis.label.set_visible(False)

ax.set_ylabel("Pupil Size \n(z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-1,3),(-0.2,0.6),1,0.2)
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_quiet_median_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###plot different dff aligned to pupil onset
plt.figure(figsize = (1.9,1.25))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","bin"]).mean().reset_index().query("type == 'dff'") ,x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "bin",palette= ["palegreen","darkgreen"],legend = None,linewidth = 1)
ax.set_xlabel("Time from \nDilation Onset (s)")
ax.set_ylabel("∆ F/F (z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-1,3),(-0.4,1.2),1,0.4)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_dff_quiet_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###plot different quartiles of pupil aligned to pupil onset
plt.figure(figsize = (0.75,0.75))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","bin"]).mean().reset_index().query("type == 'Speed'"), x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "bin",palette= ["cornflowerblue","darkblue"],legend = None,linewidth = 1)
#ax.set_xlabel("Time from \nDilation Onset (s)")
ax.xaxis.label.set_visible(False)

ax.set_ylabel("∆ F/F (z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-1,3),(-0.5,1),1,0.5)
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_pupil_quartile_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###median split z traces

id_date = {
  #"004113":["20230808"],
  "004114":["20230808","20230815"],
"004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
    
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        amplitudes = events["amplitudes"]

  
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        raw_speed = np.array(df["Running Speed"])
     
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

        move_events = get_movement_events(events,raw_speed)
        quiet_events = get_quiet_events(events,raw_speed)
        
        move_amps = []
        for i,amp in enumerate(amplitudes):
            if i in move_events:
                move_amps.append(amp)
        binned_amps = np.array(pd.qcut(move_amps,q = 2 ,labels = ["1","2"]))

        move_onsets = np.array(onsets)[np.array(move_events)]

        dil_matrix,dil_times = trial_align(move_onsets,time,pupil,fps = 20,pre = 10, post = 10 )

        for j,trial in enumerate(dil_matrix):
            baseline = np.mean(trial[170:190])
            trial = trial-baseline
            for i,Time in enumerate(dil_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Pupil","ID" : id, "Date": date,"bin":binned_amps[j]}
                df_list.append(new_dict)
                
        dff_matrix,dff_times = trial_align(move_onsets,time,dff,fps = 20,pre = 10, post = 10)
        for j,trial in enumerate(dff_matrix):
            baseline = np.mean(trial[170:190])
            trial = trial-baseline
            if j in quiet_events:
                type = "quiet"
            elif j in move_events:
                type = "running"
            else:
                type = "neither"
            for i,Time in enumerate(dff_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "dff","ID" : id, "Date": date,"bin":binned_amps[j]}
                df_list.append(new_dict)

        speed_matrix,speed_times = trial_align(move_onsets,time,speed,fps = 20,pre = 10, post = 10)

        for j,trial in enumerate(speed_matrix):
            baseline = np.mean(trial[170:190])
            trial = trial-baseline
            if j in move_events:
                type = "running"
            elif j in quiet_events:
                type = "quiet"
            else:
                type = "neither"
            for i,Time in enumerate(speed_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Speed","ID" : id, "Date": date,"bin":binned_amps[j]}
                df_list.append(new_dict)

            
        face_matrix,face_times = trial_align(move_onsets,time,face,fps = 20,pre = 10, post = 10)

        for j,trial in enumerate(face_matrix):
            baseline = np.mean(trial[170:190])
            trial = trial-baseline
            if j in move_events:
                type = "running"
            elif j in quiet_events:
                type = "quiet"
            else:
                type = "neither"
            for i,Time in enumerate(speed_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Face","ID" : id, "Date": date,"bin":binned_amps[j]}
                df_list.append(new_dict)

run_df = pd.DataFrame.from_dict(df_list)


In [ ]:
###plot different quartiles of pupil aligned to pupil onset
plt.figure(figsize = (1.9,1))
ax = sns.lineplot(data = run_df.groupby(["ID","Time from Onset (s)","type","bin"]).mean().reset_index().query("type == 'Pupil'"), x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "bin",palette= ["cornflowerblue","darkblue"],legend = None,linewidth = 1)
#ax.set_xlabel("Time from \nDilation Onset (s)")
ax.xaxis.label.set_visible(False)

ax.set_ylabel("Pupil Size \n(z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-1,3),(-0.5,2),1,0.5)
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_run_median_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###plot different dff aligned to pupil onset
plt.figure(figsize = (1.9,1.25))
ax = sns.lineplot(data = run_df.groupby(["ID","Time from Onset (s)","type","bin"]).mean().reset_index().query("type == 'dff'") ,x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "bin",palette= ["palegreen","darkgreen"],legend = None,linewidth = 1)
ax.set_xlabel("Time from \nDilation Onset (s)")
ax.set_ylabel("∆ F/F (z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-1,3),(-0.5,1.5),1,0.5)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_dff_run_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###plot different dff aligned to pupil onset
plt.figure(figsize = (1.9,1.25))
ax = sns.lineplot(data = run_df.groupby(["ID","Time from Onset (s)","type","bin"]).mean().reset_index().query("type == 'Speed'") ,x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "bin",palette= ["palegreen","darkgreen"],legend = None,linewidth = 1)
ax.set_xlabel("Time from \nDilation Onset (s)")
ax.set_ylabel("∆ F/F (z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-1,3),(-0.4,0.8),1,0.2)
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_dff_run_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###plot different dff aligned to pupil onset
plt.figure(figsize = (1.9,1.25))
ax = sns.lineplot(data = run_df.groupby(["ID","Time from Onset (s)","type","bin"]).mean().reset_index().query("type == 'Speed'") ,x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "bin",palette= ["gold","darkorange"],legend = None,linewidth = 1)
ax.set_xlabel("Time from \nDilation Onset (s)")
ax.set_ylabel("∆ F/F (z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-1,3),(-0.2,0.8),1,0.2)
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_dff_run_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###stratify within quiet dilations for facial movement

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}
#labels = ["1","2","3","4"]

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        offsets = [off/20 for off in events["offsets"]] ###get pupil offsets
        amplitudes = events["amplitudes"]
        peak_ix = events["peak_ix"]



        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        raw_speed = np.array(df["Running Speed"])

        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])
        face_deltas = []
        face_matrix,face_times = trial_align(onsets,time,face,fps = 20,pre = 10, post = 10 )
        for j,trial in enumerate(face_matrix):
          pre_face = np.mean(trial[160:190])
          post_face = np.mean(trial[200:300])
          face_deltas.append(post_face-pre_face)
        


        move_events = get_movement_events(events,raw_speed)
        quiet_events = get_quiet_events(events,raw_speed)
        quiet_amps = []
        for i,amp in enumerate(face_deltas):
            if i in quiet_events:
                quiet_amps.append(amp)
        binned_amps = np.array(pd.qcut(quiet_amps,q = 2 ,labels = ["1","2"]))



        
        for i,event in enumerate(quiet_events):
            on = np.array(events["onsets"])[event]
            off = np.array(events["offsets"])[event]
            baseline = np.mean(dff[on - 40:on - 10])
            pre_dff = np.mean(dff[on-5:on]) - baseline
            during_dff = np.mean(dff[on:off]) - baseline
            during_speed = np.mean(raw_speed[on:off])
            new_dict = { "trial" : event,"ID" : id, "Date": date,"Pre Onset ∆ F/F":pre_dff,"∆ F/F": during_dff,"amp_bin":binned_amps[i],"speed":during_speed,"amplitude":quiet_amps[i]}
            df_list.append(new_dict)

 

df = pd.DataFrame.from_dict(df_list)


In [ ]:
df

In [ ]:
plt.figure(figsize = (0.5,1.25))
g = sns.barplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "∆ F/F",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "∆ F/F",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Pupil Amplitude\n Median")
g.set_yticks(np.arange(0,0.75,0.25))
g.set(ylim=(0,0.5))
g.xaxis.label.set_visible(False)
labels = ["Low","High"]
g.set_xticklabels(labels)
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_facebin_dff_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
plt.figure(figsize = (0.5,1))
g = sns.barplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "amplitude",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "amplitude",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Pupil Amplitude\n Quartile")
# g.set_yticks(np.arange(0,2,0.5))
# g.set(ylim=(0,1))
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_facebin_dff_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stratify within run dilations

id_date = {
  #"004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        offsets = [off/20 for off in events["offsets"]] ###get pupil offsets
        amplitudes = events["amplitudes"]
        peak_ix = events["peak_ix"]



        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        raw_speed = np.array(df["Running Speed"])

        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])
        face_deltas = []
        face_matrix,face_times = trial_align(onsets,time,face,fps = 20,pre = 10, post = 10 )
        for j,trial in enumerate(face_matrix):
          pre_face = np.mean(trial[160:190])
          post_face = np.mean(trial[200:300])
          face_deltas.append(post_face-pre_face)

        move_events = get_movement_events(events,raw_speed)
        quiet_events = get_quiet_events(events,raw_speed)
        move_amps = []
        for i,amp in enumerate(face_deltas):
            if i in move_events:
                move_amps.append(amp)
        binned_amps = np.array(pd.qcut(move_amps,q = 2 ,labels = ["1","2"]))



        
        for i,event in enumerate(move_events):
            on = np.array(events["onsets"])[event]
            off = np.array(events["offsets"])[event]
            baseline = np.mean(dff[on - 40:on - 10])
            pre_dff = np.mean(dff[on-5:on]) - baseline
            during_dff = np.mean(dff[on:off]) - baseline
            during_speed = np.mean(raw_speed[on:off])
            new_dict = { "trial" : event,"ID" : id, "Date": date,"Pre Onset ∆ F/F":pre_dff,"∆ F/F": during_dff,"amp_bin":binned_amps[i],"speed":during_speed,"amplitude":move_amps[i]}
            df_list.append(new_dict)

 

df = pd.DataFrame.from_dict(df_list)


In [ ]:
plt.figure(figsize = (0.5,1.25))
g = sns.barplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "∆ F/F",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = df.groupby(["ID",'amp_bin']).mean().reset_index(), x = "amp_bin", y = "∆ F/F",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Pupil Amplitude\n Quartile")
g.set_yticks(np.arange(0,2,0.5))
g.set(ylim=(0,1))
g.xaxis.label.set_visible(False)
labels = ["Low","High"]
g.set_xticklabels(labels)
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_runbin_dff_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###median split quiet traces by facial movements

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
"004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
    
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        amplitudes = events["amplitudes"]

  
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        raw_speed = np.array(df["Running Speed"])
     
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])
        face_deltas = []
        face_matrix,face_times = trial_align(onsets,time,face,fps = 20,pre = 10, post = 10 )
        for j,trial in enumerate(face_matrix):
          pre_face = np.mean(trial[160:190])
          post_face = np.mean(trial[200:300])
          face_deltas.append(post_face-pre_face)

        move_events = get_movement_events(events,raw_speed)
        quiet_events = get_quiet_events(events,raw_speed)
        quiet_amps = []
        for i,amp in enumerate(face_deltas):
            if i in quiet_events:
                quiet_amps.append(amp)
        binned_amps = np.array(pd.qcut(quiet_amps,q = 2 ,labels = ["1","2"]))

        quiet_onsets = np.array(onsets)[np.array(quiet_events)]

        dil_matrix,dil_times = trial_align(quiet_onsets,time,pupil,fps = 20,pre = 10, post = 10 )

        for j,trial in enumerate(dil_matrix):
            baseline = np.mean(trial[170:190])
            trial = trial-baseline
            for i,Time in enumerate(dil_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Pupil","ID" : id, "Date": date,"bin":binned_amps[j]}
                df_list.append(new_dict)
                
        dff_matrix,dff_times = trial_align(quiet_onsets,time,dff,fps = 20,pre = 10, post = 10)
        for j,trial in enumerate(dff_matrix):
            baseline = np.mean(trial[170:190])
            trial = trial-baseline
            if j in quiet_events:
                type = "quiet"
            elif j in move_events:
                type = "running"
            else:
                type = "neither"
            for i,Time in enumerate(dff_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "dff","ID" : id, "Date": date,"bin":binned_amps[j]}
                df_list.append(new_dict)

        speed_matrix,speed_times = trial_align(quiet_onsets,time,speed,fps = 20,pre = 10, post = 10)

        for j,trial in enumerate(speed_matrix):
            baseline = np.mean(trial[170:190])
            trial = trial-baseline
            if j in move_events:
                type = "running"
            elif j in quiet_events:
                type = "quiet"
            else:
                type = "neither"
            for i,Time in enumerate(speed_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Speed","ID" : id, "Date": date,"bin":binned_amps[j]}
                df_list.append(new_dict)
        
        face_matrix,face_times = trial_align(quiet_onsets,time,face,fps = 20,pre = 10, post = 10)

        for j,trial in enumerate(face_matrix):
            baseline = np.mean(trial[170:190])
            trial = trial-baseline
            if j in move_events:
                type = "running"
            elif j in quiet_events:
                type = "quiet"
            else:
                type = "neither"
            for i,Time in enumerate(speed_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Face","ID" : id, "Date": date,"bin":binned_amps[j]}
                df_list.append(new_dict)

        

align_df = pd.DataFrame.from_dict(df_list)


In [ ]:
###plot different quartiles of pupil aligned to pupil onset
plt.figure(figsize = (1.9,1))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","bin"]).mean().reset_index().query("type == 'dff'"), x = "Time from Onset (s)", y = "value",errorbar = "se",hue = "bin",palette= ["cornflowerblue","darkblue"],legend = None,linewidth = 1)
#ax.set_xlabel("Time from \nDilation Onset (s)")
ax.xaxis.label.set_visible(False)

ax.set_ylabel("Pupil Size \n(z-scr)")
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,(-1,3),(-0.2,1),1,0.2)
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_quiet_median_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
# ###align to movement bouts by finding quiet regions

# ###aligned trace all trials

# id_date = {
#   "004113":["20230808"],
#   "004114":["20230808","20230815"],
#   "004115":["20230804","20230808","20230815"],
#   "004116":["20230804","20230808","20230815"],
#   "004117":["20230804","20230808","20230815"],
#   "004118":["20230804","20230808","20230815"]
# }

# df_list = []
# for id,dates in id_date.items():
#     for date in dates:
      
#         # events = get_pupil_events(id,date)
#         # onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
#         # onsets_ix = events["onsets"]
  
#         face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
#         path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
#         df = pd.read_csv(path)
#         pupil = np.array(df["Pupil Size"])
#         time = np.array(df["Time"])
#         speed = zscore(np.array(df["Running Speed"]))
#         dff = np.array(df["dFF"])
#         face = np.array(pd.read_csv(face_path)["Facial Movement"])

#         ### find quiet regions
#         N = 5
#         condition = gaussian_filter1d(face,N) > 0.05
#         quiet_regions = contiguous_regions(condition)
#         quiet_regions = [(x,y) for x,y in quiet_regions if 100 < y -x < 120]
#         run_onsets_ix = np.transpose(quiet_regions)[0]
#         run_onsets = [x/20 for x in run_onsets_ix  if (x/20) < 1790]
#         print(len(run_onsets))
        


#         dil_matrix,dil_times = trial_align(run_onsets,time,pupil,fps = 20,pre = 10, post = 20 )
#         trial_matrix,trial_times = trial_align(run_onsets,time,dff,fps = 20,pre = 10, post = 20)
#         face_matrix,face_times = trial_align(run_onsets,time,face,fps = 20,pre = 10, post = 20)
#         speed_matrix,speed_times = trial_align(run_onsets,time,speed,fps = 20,pre = 10, post = 20)

#         for j,trial in enumerate(dil_matrix):
#             baseline = np.mean(pupil[run_onsets_ix[j] - 40:run_onsets_ix[j] - 10])
#             trial = trial - baseline

#             for i,Time in enumerate(dil_times):
#                 new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Pupil","ID" : id, "Date": date}
#                 df_list.append(new_dict)

#         for j,trial in enumerate(trial_matrix):
#             baseline = np.mean(dff[run_onsets_ix[j] - 40:run_onsets_ix[j] - 10])
#             trial = trial - baseline
         
#             for i,Time in enumerate(trial_times):
#                 new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "dff","ID" : id, "Date": date}
#                 df_list.append(new_dict)

#         for j,trial in enumerate(face_matrix):
#             baseline = np.mean(face[run_onsets_ix[j] - 40:run_onsets_ix[j] - 10])
#             trial = trial - baseline
          
#             for i,Time in enumerate(face_times):
#                 new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Face","ID" : id, "Date": date}
#                 df_list.append(new_dict)

#         for j,trial in enumerate(speed_matrix):
#             baseline = np.mean(speed[run_onsets_ix[j] - 40:run_onsets_ix[j] - 10])
#             trial = trial - baseline
#             for i,Time in enumerate(speed_times):
#                 new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Speed","ID" : id, "Date": date}
#                 df_list.append(new_dict)

# align_df = pd.DataFrame.from_dict(df_list)


